<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Titanic_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!curl -o t.zip ali.140714.xyz:8000/titanic.zip
!unzip -o t.zip 
!mkdir subs
!ls 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35066  100 35066    0     0  31762      0  0:00:01  0:00:01 --:--:-- 31762
Archive:  t.zip
  inflating: data/labels.csv         
  inflating: data/test.csv           
  inflating: data/train.csv          
  inflating: data/gender_submission.csv  
data  sample_data  subs  t.zip


In [0]:
import pandas as pd
import numpy as np
# pd.set_option("display.max_columns", 9999)

def birdview(data):
    print("----------Head 5 Record----------")
    print(data.head(5))
    print("\n-----------Information-----------")
    print(data.info())
    print("\n-----------Data Types-----------")
    print(data.dtypes)
    print("\n----------Missing value-----------")
    print(data.isnull().sum())
    print("\n----------Null value-----------")
    print(data.isna().sum())
    print("\n----------Shape of Data----------")
    print(data.shape)
    
def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);

def distribution_insight(data, column):
    print("===================================")
    print("Min Value:", data[column].min())
    print("Max Value:", data[column].max())
    print("Average Value:", data[column].mean())
    print("Center Point of Data (median):", data[column].median())
    print("===================================")
    # sns.boxplot(data[column])
    

In [3]:
train = pd.read_csv('data/train.csv')
labels = train['Survived']

test = pd.read_csv('data/test.csv')
# birdview(test)
all = pd.concat([train.reset_index(drop=True), test.reset_index(drop=True)], axis=0)
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train = all.copy()
df0 = train.copy()
df0.drop(columns=['Survived', 'PassengerId', 'Cabin'], inplace=True)
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


## Encode sex and Embarked

In [6]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
df1 = df0.copy()
lb = LabelBinarizer()
sex = lb.fit_transform(train['Sex'])
df1['Sex'] = sex

"""
Fill the null value of Embarked
train[train.Pclass == 1].Embarked.value_counts() shows the most frequent Embarked place is S
train[train.Embarked.isna()]
"""
df1['Embarked'].fillna('S', inplace=True)
lb = LabelBinarizer()
_embarked = lb.fit_transform(df1['Embarked'])
lb_embarked = pd.DataFrame(_embarked, columns=('Embarked_' + c for c in lb.classes_))

df2 = pd.concat([df1.reset_index(drop=True), lb_embarked.reset_index(drop=True)], axis=1)
df2

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,S,0,0,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C,1,0,0
2,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,S,0,0,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,S,0,0,1
4,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,S,0,0,1
1305,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C,1,0,0
1306,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,0,0,1
1307,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,S,0,0,1


## Fill the age
For the age column, 177 out of 891 were missing, we're going to adopt the following strategy:
* replacing the missing age with mean value of passengers from the same class with the same sex

In [7]:
""" 
1. fill NaN age with mode of passengers from the same class and having same gender
2. categorize age by dividing each to 10
"""
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

nums = df2[['Age', 'Fare']]
imputer = IterativeImputer(random_state=0, max_iter=100)
nums = pd.DataFrame(imputer.fit_transform(nums), columns=nums.columns)

df3 = df2.copy()
df3['Age'] = nums.Age // 10
df3['Fare'] = nums.Fare
df3

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,3,"Braund, Mr. Owen Harris",1,2.0,1,0,A/5 21171,7.2500,S,0,0,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,3.0,1,0,PC 17599,71.2833,C,1,0,0
2,3,"Heikkinen, Miss. Laina",0,2.0,0,0,STON/O2. 3101282,7.9250,S,0,0,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,3.0,1,0,113803,53.1000,S,0,0,1
4,3,"Allen, Mr. William Henry",1,3.0,0,0,373450,8.0500,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",1,2.0,0,0,A.5. 3236,8.0500,S,0,0,1
1305,1,"Oliva y Ocana, Dona. Fermina",0,3.0,0,0,PC 17758,108.9000,C,1,0,0
1306,3,"Saether, Mr. Simon Sivertsen",1,3.0,0,0,SOTON/O.Q. 3101262,7.2500,S,0,0,1
1307,3,"Ware, Mr. Frederick",1,2.0,0,0,359309,8.0500,S,0,0,1


In [8]:
"""
1. Extra titles from names. E.g., Mr. Ms.
2. Label binarizer them into multiple classes
"""
import re

df4 = df3.copy()
df4['title'] = df3.Name.map(lambda n: re.compile("([A-Za-z]+)\.").search(n).group())

df4['title'] = df4['title'].replace(['Lady.', 'Capt.', 'Col.', 'Don.', 'Dr.', 'Major.', 'Rev.', 'Jonkheer.', 'Dona.'], 'Rare.')
df4['title'] = df4['title'].replace(['Countess.', 'Lady', 'Sir'], 'Royal')
df4['title'] = df4['title'].replace(['Mlle.', 'Ms.'], 'Miss.')
df4['title'] = df4['title'].replace('Mme.', 'Mrs.')
df4['title'] = df4['title'].replace('Sir.', 'Mr.')

lb = LabelBinarizer()
vecs = lb.fit_transform(df4.title)
title_1 = pd.DataFrame(vecs, columns=lb.classes_)

df5 = pd.concat([df4.reset_index(drop=True), title_1.reset_index(drop=True)], axis=1)
df5.drop(columns=['Name', 'title', 'Ticket', 'Embarked'], inplace=True)
df5

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Master.,Miss.,Mr.,Mrs.,Rare.,Royal
0,3,1,2.0,1,0,7.2500,0,0,1,0,0,1,0,0,0
1,1,0,3.0,1,0,71.2833,1,0,0,0,0,0,1,0,0
2,3,0,2.0,0,0,7.9250,0,0,1,0,1,0,0,0,0
3,1,0,3.0,1,0,53.1000,0,0,1,0,0,0,1,0,0
4,3,1,3.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
1305,1,0,3.0,0,0,108.9000,1,0,0,0,0,0,0,1,0
1306,3,1,3.0,0,0,7.2500,0,0,1,0,0,1,0,0,0
1307,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0


In [9]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler

df6 = df5.copy()
# df6.loc[df6.Fare >= 270, 'Fare'] = 270
# df6['Fare'].fillna(df6['Fare'].mode()[0], inplace = True)
df6


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Master.,Miss.,Mr.,Mrs.,Rare.,Royal
0,3,1,2.0,1,0,7.2500,0,0,1,0,0,1,0,0,0
1,1,0,3.0,1,0,71.2833,1,0,0,0,0,0,1,0,0
2,3,0,2.0,0,0,7.9250,0,0,1,0,1,0,0,0,0
3,1,0,3.0,1,0,53.1000,0,0,1,0,0,0,1,0,0
4,3,1,3.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
1305,1,0,3.0,0,0,108.9000,1,0,0,0,0,0,0,1,0
1306,3,1,3.0,0,0,7.2500,0,0,1,0,0,1,0,0,0
1307,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0


In [10]:
"""
Feature engeneering
1. extra family size from sibsp and parch
2. add alone feature
"""
df7 = df6.copy()
df7['family_size'] = 1 + df7['SibSp'] + df7['Parch']
df7['alone'] = df7['family_size'].apply(lambda x: 1 if x == 1 else 0)
df7

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Master.,Miss.,Mr.,Mrs.,Rare.,Royal,family_size,alone
0,3,1,2.0,1,0,7.2500,0,0,1,0,0,1,0,0,0,2,0
1,1,0,3.0,1,0,71.2833,1,0,0,0,0,0,1,0,0,2,0
2,3,0,2.0,0,0,7.9250,0,0,1,0,1,0,0,0,0,1,1
3,1,0,3.0,1,0,53.1000,0,0,1,0,0,0,1,0,0,2,0
4,3,1,3.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,1,1
1305,1,0,3.0,0,0,108.9000,1,0,0,0,0,0,0,1,0,1,1
1306,3,1,3.0,0,0,7.2500,0,0,1,0,0,1,0,0,0,1,1
1307,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,1,1


# Build model and train

In [0]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score

train_df = df7.loc[:890, :]
test_df = df7.loc[891:, :]
X_train, X_val, y_train, y_val = train_test_split(train_df, labels, stratify=labels, random_state=0)

In [0]:
"""
Different model might be used, thus we name submissions by
1. score
2. model name
"""
def make_predictions(model, modelname='xgboost'):
    preds = model.predict(test_df)
    y_real = pd.read_csv('data/labels.csv').Survived
    print("Test accuracy score", accuracy_score(y_real, preds))

    subs = pd.read_csv('data/gender_submission.csv')
    subs['Survived'] = preds
    export_name = str(accuracy_score(y_real, preds))[:7] + f"_{modelname}.csv"
    subs.to_csv(f"subs/{export_name}", index=False)


In [15]:
"""
Try xgboost classifier
"""
import xgboost
from sklearn.metrics import accuracy_score

model = xgboost.XGBClassifier()
model.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("Accuracy score", accuracy_score(y_val, y_preds))
make_predictions(model)

Accuracy score 0.8116591928251121
Test accuracy score 0.7631578947368421


In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_val)
print("Accuracy score", accuracy_score(y_val, y_preds))
make_predictions(lr, 'logreg')

In [0]:
"""Try random forest
"""
from sklearn.ensemble import RandomForestClassifier

parameters = {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 50, 
                  'min_samples_split': 10, 'max_features': 'sqrt', 'max_depth': 6}
    
model = RandomForestClassifier(**parameters)
model.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("Accuracy score", accuracy_score(y_val, y_preds))
make_predictions(model, 'random_forest')